In [30]:
import json
import requests
import yaml
import pandas as pd
import warnings
from bs4 import BeautifulSoup
import re
import numpy as np
from shapely.geometry import Point
warnings.simplefilter(action='ignore', category=FutureWarning)

In [5]:
#key_dict = {'key' : ''}

In [6]:
# dump the key into yaml 
with open('credentials.yml','w') as output:
    yaml.dump(key_dict, output)
    
# get the key from .yaml file
key_dict = yaml.load(open('credentials.yml'), Loader=yaml.FullLoader)  

In [23]:
key_dict = yaml.load(open("credentials.yml"), Loader=yaml.FullLoader)
gmaps_key = key_dict["key"]
api_key = gmaps_key

### Scraping Coop locations in Zürich

In [9]:
page_Coop = requests.get("https://www.profital.ch/de/flr/72139-coop-filialen/2699")
print(page_Coop.status_code) ## prints 200 as status code
#print(page.content[:3000]) ## prints the entire webpage content
soup_Coop = BeautifulSoup(page_Coop.content, "html.parser")
type(soup_Coop)

200


bs4.BeautifulSoup

In [10]:
soup_Coop.findAll('address');

In [11]:
Loc_Coop=[]
for pg in range(2,6):
    page_Coop_2 = requests.get(f"https://www.profital.ch/de/flr/72139-coop-filialen/2699?page={pg}")
    soup_Coop_2 = BeautifulSoup(page_Coop_2.content,"html.parser")
    soup_Coop_2.findAll('address')
    Loc_Coop.append(soup_Coop_2.findAll('address'))

In [12]:
Loc_Coop.append(soup_Coop.findAll('address'))
page_num_Coop=len(Loc_Coop)
page_num_Coop

5

In [13]:
Address_Coop=[]
Zip_Coop=[]
for i in range(0,page_num_Coop):
    for j in range(0,len(Loc_Coop[i])):
        Address_Coop.append(Loc_Coop[i][j].contents[0].strip())
        Zip_Coop.append(Loc_Coop[i][j].contents[2].strip().split()[0])

In [16]:
df_Coop_Locator = pd.DataFrame(list(zip(Address_Coop,Zip_Coop)), columns=["Address", "Zip_Code"])
df_Coop_Locator.head()

,Address,Zip_Code
0,Bahnhofbrücke 1,8001
1,Uetlibergstrasse 20,8045
2,Badenerstrasse 15,8004
3,Halle Löwenstrasse,8021
4,Europaallee 5,8001


In [21]:
df_Coop_Address = pd.DataFrame()
# Combine Address and Zip Code into a single column
df_Coop_Address["address"] = df_Coop_Locator["Address"] + ', ' + df_Coop_Locator["Zip_Code"]
df_Coop_Address

,address
0,"Bahnhofbrücke 1, 8001"
1,"Uetlibergstrasse 20, 8045"
2,"Badenerstrasse 15, 8004"
3,"Halle Löwenstrasse, 8021"
4,"Europaallee 5, 8001"
...,...
57,"Bärengasse 16, 8001"
58,"Alte Kalchbühlstrasse 15, 8038"
59,"Kalanderplatz 1, 8045"
60,"Bahnhofstrasse 57, 8001"


### Getting store locations from Google Maps and mapping the location to Kreis name

In [32]:
from shapely.geometry import shape, GeometryCollection, Point

In [33]:
# Function to get latitude and longitude from Google Maps API
def get_lat_lng(api_key, query):
    url = "https://maps.googleapis.com/maps/api/place/textsearch/json?"
    req = requests.get(url + "query=" + query + "&key=" + api_key)
    places_json = req.json()
    my_result = places_json.get("results", [])

    if my_result:
        location = my_result[0].get("geometry", {}).get("location", {})
        lat, lng = location.get("lat"), location.get("lng")
        return lat, lng
    else:
        return None, None

In [34]:
# Function to get kreis_name from GeoJSON data
def get_kreis_name(point, geojson_data):
    for feature in geojson_data["features"]:
        polygon = shape(feature["geometry"])
        if polygon.contains(point):
            return feature["properties"]["bezeichnung"]
    return None

In [35]:
# Load GeoJSON data
with open('data/stzh.adm_stadtkreise_a.json', 'r') as f:
    geojson_data = json.load(f)

# Create a DataFrame to store the results
columns = ["address", "latitude", "longitude", "kreis_name"]
coop_df = pd.DataFrame(columns=columns)

# Iterate through addresses in the DataFrame
for index, row in df_Coop_Address.iterrows():
    address = row["address"]
    
    # Get latitude and longitude from Google Maps API
    lat, lng = get_lat_lng(api_key, address)
    
    # If coordinates are found, create a Point object
    if lat is not None and lng is not None:
        point = Point(lng, lat)
        
        # Get kreis_name from GeoJSON data
        kreis_name = get_kreis_name(point, geojson_data)
        
        # Append results to DataFrame
        coop_df = pd.concat([coop_df, pd.DataFrame([{"address": address, "latitude": lat, "longitude": lng, "kreis_name": kreis_name}])], ignore_index=True)

# Display the final DataFrame
coop_df.head()

,address,latitude,longitude,kreis_name
0,"Bahnhofbrücke 1, 8001",47.376458,8.542357,Kreis 1
1,"Uetlibergstrasse 20, 8045",47.364747,8.520881,Kreis 3
2,"Badenerstrasse 15, 8004",47.373385,8.530586,Kreis 4
3,"Halle Löwenstrasse, 8021",47.378042,8.539067,Kreis 1
4,"Europaallee 5, 8001",47.378029,8.534917,Kreis 4


In [36]:
coop_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   address     62 non-null     object 
 1   latitude    62 non-null     float64
 2   longitude   62 non-null     float64
 3   kreis_name  62 non-null     object 
dtypes: float64(2), object(2)
memory usage: 2.1+ KB


In [37]:
# Save DataFrame to CSV
coop_df.to_csv('coop_data.csv', index=False)